In [1]:
from preprocess_utils import *

train, test = get_processed_data()
train = select_columns(train)
train.head()

/tmp/ipykernel_7067/459611240.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  train, test = get_processed_data()


,author,hindex,nb_paper,pagerank,authority,clustering_coef,n_neighbors_dist_1,mean_neighbors_dist_1,max_neighbors_dist_1,vector_coord_0,vector_coord_1
0,1964267543,4.0,5,0.000004,3.339138e-13,1.000000,5,21.800000,39,-0.186604,0.288065
1,2153592714,13.0,5,0.000002,1.433035e-17,1.000000,2,20.000000,20,-0.239073,0.446518
2,217158525,8.0,5,0.000003,6.750288e-20,1.000000,2,2.000000,2,-0.225883,0.357737
3,2123103677,11.0,3,0.000005,7.714573e-16,0.714286,7,11.833333,54,-0.174987,0.208499
4,2067710487,3.0,2,0.000004,-1.444459e-26,0.000000,2,9.841160,12,-0.170414,0.257662


In [35]:
frequency = train["hindex"].value_counts(normalize=True)

total = 0
for hindex, freq in frequency.items():
    total += freq
    if total > 0.95:
        print(hindex+1)
        break

In [37]:
large_hindex = 36

# Large Index Classification

In [33]:
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

train_split.loc[train_split["hindex"] < large_hindex, "hindex"] = 0
test_split.loc[test_split["hindex"] < large_hindex, "hindex"] = 0
train_split.loc[train_split["hindex"] >= large_hindex, "hindex"] = 1
test_split.loc[test_split["hindex"] >= large_hindex, "hindex"] = 1

print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

(130680, 11) (43561, 11)
(130680, 11)


In [34]:
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, recall_score
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print(accuracy_score(y_test,mod_preds))
print(f1_score(y_test,mod_preds))
print(recall_score(y_test,mod_preds))

0.9655655287986961
0.5639534883720931
0.44824399260628467


# Original Prediction MSE on large hindex

In [31]:
train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_big_test = [y for y in y_test if y >= large_hindex]
mod_big_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] >= large_hindex]
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_big_preds))

(130680, 11) (43561, 11)
Overall MSE: 48.50047673031719
Big Hindex MSE: 497.0707295639768


# Specific Prediction MSE on large hindex

In [40]:
big_train = train.loc[train["hindex"] >= large_hindex]

big_train_split, big_test_split = train_test_split(big_train)
print(big_train_split.shape, big_test_split.shape)

X_big_train = big_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_train = big_train_split["hindex"].to_numpy()
X_big_test = big_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_test = big_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_big_train, y_big_train)
mod_preds = model_cat.predict(X_big_test)
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_preds))

(6469, 11) (2157, 11)
Big Hindex MSE: 170.16960484061605


# Original Prediction MSE on small hindex

In [43]:
train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_small_test = [y for y in y_test if y < large_hindex]
mod_small_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] < large_hindex]
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_small_preds))

(130680, 11) (43561, 11)
Overall MSE: 51.41873596298948
Small Hindex MSE: 25.906271543186012


# Specific Prediction MSE on small hindex

In [44]:
small_train = train.loc[train["hindex"] < large_hindex]

small_train_split, small_test_split = train_test_split(small_train)
print(small_train_split.shape, small_test_split.shape)

X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_small_train, y_small_train)
mod_preds = model_cat.predict(X_small_test)
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_preds))

(124211, 11) (41404, 11)
Small Hindex MSE: 18.99344045082073


# Combine Specific prediction and Large Index Classification

In [73]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score

large_hindex = 6

train_split, test_split = train_test_split(train)
train_split_classifier, test_split_classifier = train_split.copy(), test_split.copy()
print(train_split.shape, test_split.shape)

large_train_split = train_split.loc[train_split["hindex"] >= large_hindex]
large_test_split = test_split.loc[test_split["hindex"] >= large_hindex]
small_train_split = train_split.loc[train_split["hindex"] < large_hindex]
small_test_split = test_split.loc[test_split["hindex"] < large_hindex]

train_split_classifier.loc[train_split_classifier["hindex"] < large_hindex, "hindex"] = 0
train_split_classifier.loc[train_split_classifier["hindex"] >= large_hindex, "hindex"] = 1
test_split_classifier.loc[test_split_classifier["hindex"] < large_hindex, "hindex"] = 0
test_split_classifier.loc[test_split_classifier["hindex"] >= large_hindex, "hindex"] = 1

# Balance small and large data
# large_train_split_temp = large_train_split.append(small_train_split.sample(n=int(len(small_train_split) / 3), ignore_index=True))
# small_train_split = small_train_split.append(large_train_split_temp.sample(n=int(len(large_train_split) / 1.3), ignore_index=True))
# large_train_split = large_train_split_temp

print(large_train_split.shape, small_train_split.shape)

# Data for the classifier
X_train_classifier = train_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train_classifier = train_split_classifier["hindex"].to_numpy()
X_test_classifier = test_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test_classifier = test_split_classifier["hindex"].to_numpy()

# Data for large hindex regressor
X_large_train = large_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_train = large_train_split["hindex"].to_numpy()
X_large_test = large_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_test = large_test_split["hindex"].to_numpy()

# Data for small hindex regressor
X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

# Original Data
X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()


model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Classical Hindex MSE:", mean_squared_error(y_test, preds))
print("Classical Hindex MSE on small:", mean_squared_error(y_small_test, model.predict(X_small_test)))
print("Classical Hindex MSE on large:", mean_squared_error(y_large_test, model.predict(X_large_test)))


model_small = CatBoostRegressor(verbose=False)
model_small.fit(X_small_train, y_small_train)
small_preds = model_small.predict(X_test)
print("Small Hindex MSE:", mean_squared_error(y_test, small_preds))
print("Small Hindex MSE on small:", mean_squared_error(y_small_test, model_small.predict(X_small_test)))
print("Small Hindex MSE on large:", mean_squared_error(y_large_test, model_small.predict(X_large_test)))


model_large = CatBoostRegressor(verbose=False)
model_large.fit(X_large_train, y_large_train)
large_preds = model_large.predict(X_test)
print("Large Hindex MSE:", mean_squared_error(y_test, large_preds))
print("Large Hindex MSE on small:", mean_squared_error(y_small_test, model_large.predict(X_small_test)))
print("Large Hindex MSE on large:", mean_squared_error(y_large_test, model_large.predict(X_large_test)))


hindex_classifier = CatBoostClassifier(verbose=False, num_trees=2000)
hindex_classifier.fit(X_train_classifier, y_train_classifier)
hindex_classifier_preds = hindex_classifier.predict(X_test_classifier)
print(accuracy_score(y_test_classifier, hindex_classifier_preds), f1_score(y_test_classifier,hindex_classifier_preds), recall_score(y_test_classifier,hindex_classifier_preds))

combine_preds = []
for i in range(len(y_test)):
    if hindex_classifier_preds[i] == 0:
        combine_preds.append(small_preds[i])
    else:
        combine_preds.append(large_preds[i])
print("Combine Hindex MSE:", mean_squared_error(y_test, combine_preds))

(130680, 11) (43561, 11)
(66184, 11) (64496, 11)
Classical Hindex MSE: 48.63500832797908
Classical Small Hindex MSE: 10.36615628435076
Classical Large Hindex MSE: 86.82136926694538
Small Hindex MSE: 188.14152413363985
Small Hindex MSE on small: 0.8738218202998101
Small Hindex MSE on large: 375.005558220566
Large Hindex MSE: 72.11524054180073
Large Hindex MSE on small: 61.47580263670916
Large Hindex MSE on large: 82.73174441728581
0.8657973875714515 0.8665662375604857 0.8706200697119795
Combine Hindex MSE: 51.477366180864685
